In [1]:
import cv2
import sys
import numpy as np

### blur

In [3]:
src = cv2.imread("../data/rose.bmp", cv2.IMREAD_GRAYSCALE)

dst = cv2.blur(src, (3,3))

cv2.imshow("src", src)
cv2.imshow("dst", dst)
cv2.waitKey()

cv2.destroyAllWindows()

### blur kernel값 변경하면서 확인해보기

In [9]:
src = cv2.imread("../data/rose.bmp", cv2.IMREAD_GRAYSCALE)
cv2.imshow("src", src)

blurred_list = []

for ksize in (3, 5, 7):
    dst = cv2.blur(src, (ksize, ksize))

    # 설명 텍스트 추가
    desc = f'Mean: {ksize}x{ksize}'
    dst_color = cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR)  # putText는 컬러에서 잘 보여
    cv2.putText(dst_color, desc, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                1.0, (0, 0, 255), 2, cv2.LINE_AA)

    blurred_list.append(dst_color)

# 가로로 이미지 이어붙이기
combined = cv2.hconcat(blurred_list)

cv2.imshow("All Mean Blurs", combined)
cv2.waitKey()

cv2.destroyAllWindows()


### 가우시안 블러 함수

In [13]:
src = cv2.imread("../data/rose.bmp", cv2.IMREAD_GRAYSCALE)
cv2.imshow("src", src)

for sigma in range(1, 6):
    dst = cv2.GaussianBlur(src, (0, 0), sigma)

    # 텍스트 표시용 컬러 이미지로 변환
    dst_color = cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR)
    desc = f'sigma = {sigma}'
    cv2.putText(dst_color, desc, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                1.0, (0, 0, 255), 2, cv2.LINE_AA)

    # 창 이름 다르게 해서 모두 띄우기
    win_name = f'Gaussian Blur sigma={sigma}'
    cv2.imshow(win_name, dst_color)

cv2.waitKey()
cv2.destroyAllWindows()

### 샤프닝 필터 - 흑백

In [15]:
src = cv2.imread("../data/rose.bmp", cv2.IMREAD_GRAYSCALE)

blr = cv2.GaussianBlur(src, (0, 0), 2)
dst = np.clip(2.0*src - blr, 0, 255).astype(np.uint8)

cv2.imshow("src", src)
cv2.imshow("blr", blr)
cv2.imshow("dst", dst)
cv2.waitKey()

cv2.destroyAllWindows()

### 샤프닝필터 - 컬러

In [18]:
src = cv2.imread("../data/rose.bmp")

src_ycrcb = cv2.cvtColor(src, cv2.COLOR_BGR2YCrCb)

src_f = src_ycrcb[:, :, 0].astype(np.float32)
blr = cv2.GaussianBlur(src_f, (0, 0), 2.0)
src_ycrcb[:, :, 0] = np.clip(2. * src_f - blr, 0, 255).astype(np.uint8)

dst = cv2.cvtColor(src_ycrcb, cv2.COLOR_YCrCb2BGR)

cv2.imshow("src", src)
cv2.imshow("dst", dst)
cv2.waitKey()

cv2.destroyAllWindows()

### 메디안 블러

In [2]:
src = cv2.imread("../data/noise.bmp", cv2.IMREAD_GRAYSCALE)

dst = cv2.medianBlur(src, 3)

cv2.imshow("src", src)
cv2.imshow("dst", dst)
cv2.waitKey()

cv2.destroyAllWindows()

### 양방향(좌표공간, 색공간) 필터

In [6]:
src = cv2.imread("../data/lenna.bmp")
dst = cv2.bilateralFilter(src, -1, 10, 5)

cv2.imshow("src", src)
cv2.imshow("dst", dst)
cv2.waitKey()

cv2.destroyAllWindows()

### 카툰필터 카메라, 스케치 카메라

In [9]:
def cartoon_filter(img):
    h, w = img.shape[:2]
    img2 = cv2.resize(img, (w//2, h//2))

    blr = cv2.bilateralFilter(img2, -1, 20, 7)
    edge = 255 - cv2.Canny(img2, 80, 120)
    edge = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)

    dst = cv2.bitwise_and(blr, edge)
    dst = cv2.resize(dst, (w, h), interpolation=cv2.INTER_NEAREST)

    return dst

def pencil_sketch(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blr = cv2.GaussianBlur(gray, (0,0), 3)
    dst = cv2.divide(gray, blr, scale=255)
    return dst
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("video open failed!")
    sys.exit()

cam_mode = 0

while True:
    ret, frame = cap.read()

    if not ret:
        break

    if cam_mode == 1:
        frame = cartoon_filter(frame)
    elif cam_mode == 2:
        frame = pencil_sketch(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)

    cv2.imshow("frame", frame)
    key = cv2.waitKey(1)

    if key == 27:
        break
    elif key == ord(" "):
        cam_mode += 1
        if cam_mode == 3:
            cam_mode = 0

cap.release()
cv2.destroyAllWindows()